In [1]:
import  dlc_bci as bci
import torch
from torch import optim
from torch import Tensor
from torch.autograd import Variable
from torch import nn
import numpy as np
import random
from torch.nn import functional as F
from sklearn.linear_model import LogisticRegression

In [ ]:
m = nn.Threshold(0.1, 20)
input_ = torch.randn(2)
print(input_)
m.forward(Variable(input_))


In [105]:
train_input, train_target = bci.load(root =  './data', one_khz=False)
print(str(type(train_input)), train_input.size())
print(str(type(train_target)), train_target.size())
test_input , test_target = bci.load(root =  './data', train = False)
print(str(type(test_input)), test_input.size())
print(str(type(test_target)), test_target.size())

<class 'torch.FloatTensor'> torch.Size([316, 28, 50])
<class 'torch.LongTensor'> torch.Size([316])
<class 'torch.FloatTensor'> torch.Size([100, 28, 50])
<class 'torch.LongTensor'> torch.Size([100])


In [ ]:
def up_sampled(data_input, data_target):
    custom_train_input = torch.Tensor()
    custom_train_target = torch.LongTensor()
    perm = torch.randperm(500)
    data_input = data_input[:,:,perm]
    for i in range(10):
        custom_train_input = torch.cat((custom_train_input, data_input.narrow(2,i*50,50)),0)
    
    for i in range(train_target.size(0)):
        custom_train_target = torch.cat((custom_train_target, torch.LongTensor(10).fill_(data_target[i])),0)
    print(custom_train_input.size())   
    return custom_train_input, custom_train_target
    



In [ ]:
train_input, train_target = up_sampled(train_input, train_target)
index = torch.LongTensor(random.sample(range(3160),316))
train_input = torch.index_select(train_input, 0, index)
train_target = torch.index_select(train_target, 0, index)
train_input.size()

In [ ]:
train_input = train_input.numpy().reshape(316,-1)
train_target = train_target.numpy()
test_input = test_input.numpy().reshape(100,-1)
test_target = test_target.numpy()

train_input.shape

In [ ]:
train_input.size()

In [ ]:
lr = LogisticRegression()
lr.fit(train_input, train_target)

lr.score(test_input, test_target)

In [106]:
mean, std = train_input.mean(), train_input.std()
train_input.sub_(mean).div_(std)
test_input.sub_(mean).div_(std)

train_input = train_input.view(train_input.size(0),1,28,-1)
test_input = test_input.view(test_input.size(0),1,28,-1)

#train_input = train_input.view(train_input.size(0),-1)
#test_input = test_input.view(test_input.size(0),-1)

train_input, train_target = Variable(train_input), Variable(train_target)
test_input, test_target = Variable(test_input), Variable(test_target)


print(train_input.size())


torch.Size([316, 1, 28, 50])


In [108]:
class Net(nn.Module):
    def __init__(self, nb_hidden):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=5)
        self.fc1 = nn.Linear(64*2*5, nb_hidden)
        self.fc2 = nn.Linear(nb_hidden, 2)

    def forward(self, x):
        x = F.dropout2d(F.tanh(F.max_pool2d(self.conv1(x), kernel_size=3, stride=3)), 0.75)
        x = F.dropout2d(F.tanh(F.max_pool2d(self.conv2(x), kernel_size=2, stride=2)))
        #print(x.size())
        
        x = F.dropout(F.tanh(self.fc1(x.view(-1, 64*2*5))))
        x = F.dropout(self.fc2(x))
        return x

In [114]:
def train_model(model, train_input, train_target, mini_batch_size, lambda_=0.1):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters())
    nb_epochs = 50

    for e in range(0, nb_epochs):
        sum_loss = 0
        for b in range(0, train_input.size(0), mini_batch_size):
            train = train_input.narrow(0, b, mini_batch_size)
            m,s = train.mean(), train.std()
            train = train.sub(m).div(s)
            output = model(train_input.narrow(0, b, mini_batch_size))
            loss = criterion(output, train_target.narrow(0, b, mini_batch_size))
            model.zero_grad()
            loss.backward()
            optimizer.step()
            sum_loss += loss.data[0]
        print(e,sum_loss)

def compute_nb_errors(model, data_input, data_target, mini_batch_size):

    nb_data_errors = 0

    for b in range(0, data_input.size(0), mini_batch_size):
        output = model(data_input.narrow(0, b, mini_batch_size))
        _, predicted_classes = torch.max(output.data,1)
        for k in range(0, mini_batch_size):
            if data_target.data[b + k] != predicted_classes[k]:
                nb_data_errors = nb_data_errors + 1

    return nb_data_errors


In [ ]:
def create_deep_model():
    return nn.Sequential(
        nn.Linear(2, 4),
        nn.ReLU(),
        nn.Linear(4,2)
       
        
    )

In [115]:
model = Net(350)
#model = create_deep_model()
train_model(model, train_input, train_target,4)

0 65.68065404891968
1 55.663880467414856
2 55.01844197511673
3 54.862157106399536
4 54.18143540620804
5 52.17375400662422
6 47.47603453695774
7 39.68996595218778
8 39.36400820687413
9 30.724564619362354
10 24.536316198296845
11 19.127312743337825
12 14.248550981283188
13 9.1957681364147
14 7.005348525563022
15 9.990702359733405
16 4.128785147622693
17 2.0476230320637114
18 0.6315887164637388
19 0.10596043579243997
20 0.04877681579455384
21 0.038765786772273714
22 0.03230430736311973
23 0.02767772950937797
24 0.024138525455782656
25 0.021322480359231122
26 0.01903982218937017
27 0.01716194474738586
28 0.015580042410419992
29 0.014225232780518127
30 0.013054622811068839
31 0.012025282971990237
32 0.011112709014923894
33 0.010298874856744078
34 0.009568590958906498
35 0.008908053144750738
36 0.008311660039453272
37 0.0077705193273232
38 0.007276926009126328
39 0.006823488236932462
40 0.006407289231447066
41 0.006025224808581697
42 0.0056730087542291585
43 0.005347176667100939
44 0.0050449

In [116]:
print(100 - compute_nb_errors(model, train_input, train_target, 4) / train_input.size(0) * 100)
100 - compute_nb_errors(model, test_input, test_target, 4) / test_input.size(0) * 100

100.0


74.0

In [ ]:
x = Variable(Tensor([1,2,2]), requires_grad=)
l = x.norm()
l.backward()
x.grad